# Quantum Variational Simulator

The problem of simulating quantum many-body systems is well-known to be extremely challenging for classical computers. This is due to the exponential dimension of the Hilbert space associated to these systems. For these systems, it is impossible to diagonalize the Hamiltonian to find the ground state energy, or to solve the Schrödinger equation to obtain the evolution of the system. Variational Classical Simulation (VCE) is an alternative to these direct methods because effectively reduces the dimension of the systems using efficient classical representations. It can be used to find the ground state and to simulate time evolution of quantum systems. However, for highly-entangled systems VQE is usually infeasible, because there are not good classical representations of these states. 

The efficient alternative is using quantum systems to simulate quantum systems. The Trotterization technique is the usual method to perform quantum evolution, and approximates the time evolution under a Hamiltonian $H = \sum_j H_j$ from $0$ to $T$ by

$$
e^{-i H T} \approx \prod_{n=1}^{N}  \left( \prod_j e^{-i H_j \Delta t}\right),
$$

where $e^{-i H_j \Delta t}$ is the unitary generated by $H_j$ for a short time $\Delta t = T/N$. Variational Quantum Algorithms (VQA) are the quantum version of VCE and they can be used to find the ground state energy of relevant Hamiltonians using efficient variational forms and classical optimization. These algorithms can also be used as an alternative for Trotterization, that usually requires circuits with many quantum operations a large depth. Which is not desirable in the current era of quantum computation, because quantum processors are still noisy and the depth of the circuits is restricted by the decoherence time of the system. 

Here, we reproduced the results of the papers "Efficient Variational Quantum Simulator Incorporating Active Error Minimization" and "Theory of Variational Quantum Simulation" for quantum Isings models of 2 and 3 qubits. 

## The time-dependent variational principle

This principle corresponding to the Schrödinger equation reads $\delta \int_{t_{i}}^{t_{f}} dt L=0$, where the Lagrangian is $L=\langle\Psi(t)|\left(i \frac{\partial}{\partial t}-H\right)| \Psi(t)\rangle$. Assuming that the state $| \Psi(t)\rangle$ is determined by a set of real parameters $\left\{\lambda_{k}(t)\right\}$, that is $|\Psi(t)\rangle \equiv\left|\Psi\left(\lambda_{1}, \lambda_{2}, \ldots\right)\right\rangle$. The the Lagrangian can be rewritten as 

$$
L=i \sum_{k}\langle\Psi| \frac{\partial|\Psi\rangle}{\partial \lambda_{k}} \dot{\lambda}_{k}-\langle\Psi|H| \Psi\rangle
$$

which is a function of parameters $\{\lambda_{k}\}$ and their time derivatives $\left\{\dot{\lambda}_{k}=\left[d \lambda_{k} / d t\right]\right\}$. After applying the Euler-Lagrange equation, the evolution of parameters is

$$
\sum_{q} A_{k, q} \dot{\lambda}_{q}=V_{k}
$$

where

$$
A_{k, q}=i  \frac{\partial\langle\Psi|}{\partial \lambda_{k}} \frac{\partial|\Psi\rangle}{\partial \lambda_{q}}+\text { H.c. } \quad \text{and} \quad V_{k}= \frac{\partial\langle\Psi|}{\partial \lambda_{k}} H|\Psi\rangle+\text { H.c. }
$$

## Variational algorithm on a hybrid computer